In [1]:
from fytok.Tokamak import Tokamak

2024-01-05 14:25:24,900 [    spdm]     INFO: 
###################################################################################################

    ______      _____     _
   / ____/_  __|_   _|__ | | __
  / /_  / / / /  | |/ _ \| |/ /
 / __/ / /_/ /   | | (_) |   <
/_/    \__, /    |_|\___/|_|\_\
      /____/

 Copyright (c) 2021-present Zhi YU (Institute of Plasma Physics Chinese Academy of Sciences)
            
 url: https://gitee.com/openfusion/fytok_tutorial 
      https://github.com/fusion-yun/fytok_tutorial

 version = alpha  (spdm=beta extension=0.2.2.dev19+g14adade.d20231205, imas_version=v3_38_1_dirty)

 Run by salmon at 2024-01-05T14:25:24.700635.
 Job ID: fytok_salmon_surface_13363

###################################################################################################



In [2]:
tok = Tokamak("mdsplus:///home/salmon/workspace/fytok_data/mdsplus/~t/?enable=efit_east", device="east", shot=70754)

2024-01-05 14:25:25,127 [    spdm]     INFO: Load module spdm.plugins.data.plugin_xml


In [3]:
tok.refresh(time=5.0)

2024-01-05 14:25:25,154 [    spdm]     INFO: Initialize module fytok.modules.CoreProfiles 


AttributeError: 'CoreProfiles' object has no attribute 'inputs'

In [ ]:
display(tok)

In [ ]:
tok.equilibrium.time_slice.current.profiles_2d.psi

In [ ]:
tok.wall

In [ ]:
tok.pf_active

In [ ]:
tok.magnetics

In [ ]:
from spdm.view import View as sp_view

fig=sp_view.display(tok)